# 뉴스 크롤링

In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

In [ ]:
!apt-get update
!apt install chromium-chromedriver

In [41]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)

In [96]:
# 카테고리 별 100개 기사 크롤링 (링크)
# 한 페이지에 20개

url_list = []   # url 저장 리스트
a_list = []     # a 태그 저장 리스트

for category in range(1):     # 6
    for page in range(1, 2):  # 1, 6
        url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={100 + category}#&date=%2000:00:00&page={page}'
        browser.get(url)

        time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")
        a_list.extend(soup.select(".type06_headline dt+dt a"))
        a_list.extend(soup.select(".type06 dt+dt a"))

        print("카테고리 : ", 100 + category, "페이지 : ", page)

for a in a_list:
    url_list.append(a["href"])  # 링크

카테고리 :  100 페이지 :  1


In [ ]:
url_list

In [98]:
# 수집한 링크의 제목, 본문 크롤링
# + 날짜 추가하기

title_list = []
content_list = []
title = []
content = []

for url in url_list:

      browser.get(url)

      time.sleep(1)

      soup = BeautifulSoup(browser.page_source, "html.parser")

      title.extend(soup.select("#title_area span"))
      content.extend(soup.select("#dic_area"))

for t in title:
  title_list.append(t.text)

for c in content:
  c = c.text.replace('\n', '')
  c = c.replace('\xa0', '')
  c = c.replace('\t', '')
  content_list.append(c)

In [100]:
# 데이터 프레임으로 변환

article_df = pd.DataFrame({"제목" : title_list,
                          "본문" : content_list,
                           "링크" : url_list})

article_df

,제목,본문,링크
0,"'개딸' 행패에 비명계 폭발…이상민 ""이재명 수수방관하나""",이재명 더불어민주당 대표가 지난 9일 오후 서울 중랑구 녹색병원에서 퇴원하며 지지자...,https://n.news.naver.com/mnews/article/015/000...
1,100조 오일머니 잡고 현안 목소리 낸 韓…힘 실리는 중동 외교,윤석열 대통령이 4박 6일간의 사우디아라비아와 카타르 순방을 통해 중동과 경제 ...,https://n.news.naver.com/mnews/article/025/000...
2,"尹 “‘박정희 혜안’ 공부해, 한국 재도약 해야”…‘보수 결집’ 움직임","박정희 前대통령 44주기 추도식윤석열 대통령, 박정희 전 대통령 추도식 윤석열 대통...",https://n.news.naver.com/mnews/article/020/000...
3,"""이재명 정치생명 연장 재판부""…""정권 돌격대 감사원""(종합)","[국감초점] 국회 법사위 종감…여, 전현희 의혹 ""사실무근""야 ""감사원 정적죽이기에...",https://n.news.naver.com/mnews/article/421/000...
4,"교육위, '의대 정원·마약 처방' 쟁점…""이주호 왕따 수준"" ""의사가 마약 공급""(종합)","유기홍 ""의대 정원 용두사미로 끝날라…교육부도 역할해야""조경태 ""의사가 연예인 마약...",https://n.news.naver.com/mnews/article/003/001...
5,"박용우 위니아 회장 ""임금체불, 사과…자산 팔아 변제할 것""",[the300][2023 국정감사]박영우 대유위니아그룹 회장이 26일 오후 서울 여...,https://n.news.naver.com/mnews/article/008/000...
6,"홍석준 ""文 집권 후 KBS1 라디오 진행자 좌파로 꾸려져…국민 신뢰도 추락""","""공영방송에 안 맞는 편향 방송""""野 친화적견해, 정부·여당보다최소 10배 정도는 ...",https://n.news.naver.com/mnews/article/119/000...
7,"""기대할 게 있겠냐"" ""나름 신선""…인요한 혁신위 인선 당내 반응은","""혁신위에 관심 없어…대부분 뜨악하는 분위기""""국민이 보기엔 부족할 수 있지만 다양...",https://n.news.naver.com/mnews/article/421/000...
8,"유인촌 ""관광공사 이재환 부사장 잘못한 일 있으면 책임져야""","'나는 낙하산' 발언 직장내 괴롭힘 의혹김장실 사장 ""감사 결과 따라 조치""▲ 유인...",https://n.news.naver.com/mnews/article/654/000...
9,"심상찮은 TK 지지율…尹대통령, 귀국 직후 朴 만났다 [용산실록]",박정희 전 대통령 서거 제44주기 추도식에 참석한 윤석열 대통령과 박근혜 전 대통령...,https://n.news.naver.com/mnews/article/016/000...


# 전처리

**형태소란, 일정한 뜻(의미)을 가진 가장 작은 말의 단위를 말한다. 형태소는 자립성의 유무와 실질적 의미의 유무에 따라 자립 형태소, 의존 형태소, 실질 형태소, 형식 형태소로 나뉜다.**